In [1]:
import os
import cv2 as cv
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras.applications import ResNet50 , ResNet101, ResNet152

In [ ]:
###################################################### 실행 안함. 자동완성용

from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
######################################################

In [2]:

folder_list = os.listdir('/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train')
folder_list.remove('.DS_Store') 

data = []
pic_count = []

for folder_name in folder_list:
    folder_path = '/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train/' + folder_name
  
    image_list = os.listdir(folder_path)
    list_len = len(image_list)
    pic_count.append(list_len)
    
    for image_name in image_list:
        image_path = folder_path +'/' + image_name
        img = cv.imread(image_path)
        resize = cv.resize(img, (400,400), interpolation=cv.INTER_CUBIC)
        gray = cv.cvtColor(resize, cv.COLOR_BGR2GRAY)
        threshold, thresh = cv.threshold(gray,150, 255, cv.THRESH_BINARY)
        data.append(thresh)

In [3]:
answer = []
for idx, nm in enumerate(folder_list):
        answer += [nm] * pic_count[idx]
answer = np.array(answer)
answer = pd.get_dummies(answer)
answer = answer.values

In [4]:
answer


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False]])

In [5]:
xtrain, xtest, ytrain, ytest = train_test_split(np.array(data),answer,test_size=0.2, shuffle=True)

In [6]:
print(xtrain.shape)
print(xtest.shape)

(2765, 400, 400)
(692, 400, 400)


In [7]:
xtrain = xtrain.reshape(2765,400,400,1).astype('float32')/255
xtest = xtest.reshape(692,400,400,1).astype('float32')/255

In [8]:
resnet50 = ResNet50(weights=None, include_top=None, input_shape=(400,400,1))

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [9]:
model = Sequential()
model.add(resnet50)
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(19, activation = 'softmax'))

model.summary()
model.compile(optimizer= 'adam', loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 13, 13, 2048)      23581440  
                                                                 
 flatten (Flatten)           (None, 346112)            0         
                                                                 
 dropout (Dropout)           (None, 346112)            0         
                                                                 
 dense (Dense)               (None, 19)                6576147   
                                                                 
Total params: 30,157,587
Trainable params: 30,104,467
Non-trainable params: 53,120
_________________________________________________________________


In [10]:
modelpath="resnet50_bestmodel.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

In [11]:
history = model.fit(xtrain,ytrain, validation_data=(xtest, ytest), batch_size=50, epochs = 500, verbose=1, callbacks=[early_stopping_callback,checkpointer])

Epoch 1/500


2023-05-10 15:27:38.324112: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


56/56 [==============================] - 402s 7s/step - loss: 47.4715 - accuracy: 0.3020 - val_loss: 32536.4336 - val_accuracy: 0.1445
Epoch 2/500
56/56 [==============================] - 388s 7s/step - loss: 9.7052 - accuracy: 0.2879 - val_loss: 15.2733 - val_accuracy: 0.1301
Epoch 3/500
56/56 [==============================] - 412s 7s/step - loss: 2.1553 - accuracy: 0.3975 - val_loss: 2.2296 - val_accuracy: 0.3194
Epoch 4/500
47/56 [========================>.....] - ETA: 1:10 - loss: 2.0083 - accuracy: 0.3868

KeyboardInterrupt: 

In [18]:
my_model = load_model('resnet50_bestmodel.hdf5')

In [19]:
score = my_model.evaluate(xtest, ytest)
print('Test accuracy:', score[1])

22/22 [==============================] - 1s 39ms/step - loss: 2.9381 - accuracy: 0.4104
Test accuracy: 0.41040462255477905
